In [2]:
import pandas as pd
import numpy as np
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from IPython.display import clear_output as clear
import re
import operator
import itertools as it
import csv
#import vzta as ta
import warnings
from mitie import mitie
from os import listdir
from os.path import isfile, join,splitext

from pandas import ExcelWriter
warnings.filterwarnings("ignore")
from __future__ import print_function

try:
    nlp
except:
    nlp = spacy.load('en')

    
#ner_model ="C:\\Users\\v617346\\AppData\\Local\\Continuum\\Anaconda2\\Lib\\site-packages\\mitie\MITIE-models\\english\\ner_model.dat"

#try:
#    NER
#except:
 #   NER=mitie.named_entity_extractor(ner_model)



In [22]:
PHONE_REGEX = re.compile(r'(?:^|(?<=[^\w)]))(\+?1[ .-]?)?(\(?\d{3}\)?[ .-]?)?\d{3}[ .-]?\d{4}(\s?(?:ext\.?|[#x-])\s?\d{2,6})?(?:$|(?=\W))')
phone_re=re.compile("(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})")
filename ="C:\\Users\\v617346\\Documents\\MyJabberFiles\\ama.dankwah@verizon.com\\App_Website_data.txt"
def docsFromText(filename, sep=',',text_col_index=0,skipHeader=False):
    inFile= open(filename)
    if skipHeader:
        inFile.readline()
    for l in inFile:
        fields=l.split(sep)
        text=fields[text_col_index]
        yield text

text ="2. Enter your PIN number: (530) 443-2359"

phones=[]
tweets=[]
for text in docsFromText(filename):
    text= text.decode('utf-8').strip()
    result=phone_re.findall(text)
    if result:
        for r in result:
            phones.append(r)
            tweets.append(text)

#print(phones)

data = {'tweet': tweets,'phone': phones}
dframe = pd.DataFrame(data, columns=data.keys())
dframe.to_excel(r'C:\\Users\\v617346\\Documents\\Jupyter\\Social\\data\\Social_phone_numbers.xlsx', index=False)

In [3]:
def getNer(text):
    if len(text)>0:
       
        tokens =mitie.tokenize(txt)
        entities = NER.extract_entities(tokens)
        ents=[]
        for e in entities:
            entity= ' '.join( [ tokens[i] for i in e[0]] ) + '||'+e[1]
            entity=entity.decode('utf-8')
            ents.append(unicode(entity))
        return ents
    return []


def special_ngrams(doc, n,
           filter_stops=True, filter_punct=True, filter_nums=False,
           include_pos_tags=None, exclude_pos_tags=None, min_freq=1):
   
    if n < 1:
        raise ValueError('n must be greater than or equal to 1')


    ngrams_ = (doc[i: i + n]
               for i in range(len(doc) - n + 1))
    ngrams_ = (ngram for ngram in ngrams_
               if not any(w.is_space for w in ngram))
    if filter_stops is True:
        ngrams_ = (ngram for ngram in ngrams_
                   if not stopGram(ngram))
                   #if not ngram[0].is_stop and not ngram[-1].is_stop)
  
    if filter_punct is True:
        ngrams_ = (ngram for ngram in ngrams_
                   #if not isPunct(ngram))
                   if not any(w.is_punct for w in ngram))
   
    if filter_nums is True:
        ngrams_ = (ngram for ngram in ngrams_
                   if not any(w.like_num for w in ngram))
    if include_pos_tags:
        ngrams_ = (ngram for ngram in ngrams_
                   if all(w.pos_ in include_pos_tags for w in ngram))
    if exclude_pos_tags:
        ngrams_ = (ngram for ngram in ngrams_
                   if not any(w.pos_ in exclude_pos_tags for w in ngram))
    if min_freq > 1:
        ngrams_ = list(ngrams_)
        freqs = itertoolz.frequencies(normalized_str(ngram) for ngram in ngrams_)
        ngrams_ = (ngram for ngram in ngrams_
                   if freqs[normalized_str(ngram)] >= min_freq)

    for ngram in ngrams_:
        yield ngram
        

ignore_words=["ok","thank","thanks","yes","oh","great","hi","hello",
              "'m","'s","'ll","'ve","okay","'d",
              "'re","ass","lol","think",'ill','want','like',
             'sorry','ve','guys','wo','yeah','right','ca','jessica','mike','amy']

 

def stopGram( ngram,n=3):
    if n==1 and ngram[0].is_stop:
        return True
    if len([i for i in ngram if i.is_stop]) > 1:
        return True
        
    return False

def isPunct( ngram,n=3):
    if n==1 and ngram[0].is_punct:
        return True
    n_puncts= len([i for i in ngram if i.is_punct])
    if n_puncts > 1:
        return True
    if not ngram[-1].is_punct  and n_puncts >0 :
        return True
    return False


def topTerms(matrix,id2term,n=100,cols=[]):
    ids = range(len(id2term))
    ts= {id2term[i]:matrix[:,i].sum(axis=0)[0,0] for i in ids}
    sorted_ts = sorted(ts.iteritems(), key=operator.itemgetter(1), reverse=True)[:n]
    if cols:
        colnames=cols
    else:
        colnames=['Term','Count']
    return pd.DataFrame([ (t[0],t[1]) for t in sorted_ts],columns=colnames)
    
def discriminator(terms1,terms2,names):
    b = []
    b.extend(it.repeat(True, times=len(terms1)))
    b.extend(it.repeat(False, times=len(terms2)))
    all_terms =terms1 + terms2
    print(len(b))
    r= ([],[]) #ta.keyterms.most_discriminating_terms(all_terms,b,max_n_terms=1000, top_n_terms=100)
    
    g1len= len(r[0])
    g2len= len(r[1])
    lmax = max(g1len, g2len)
    combined =[];
    for i in range(0,lmax-1):
        a=''
        b=''
        if i < g1len:
            a= r[0][i]
        if i < g2len: 
            b= r[1][i]
        combined.append((a,b))
    return pd.DataFrame(combined, columns=names)
 
def ne(doc):
    nes=[]
    current
    for ent in doc.ents:
        if len(ent.text) > 1:
            text= ent.text.lower()
            typ =' '.join([e.ent_type_ for e in ent])
            nes.append(text +'||' + typ)
        
            
    return nes

 
def ne(doc):
    nes=[]
    ents=[]
    for e in doc.ents:
        ents.append(e)
    for ent in doc.ents:
        if len(ent.text) > 1:
            text= ent.text.lower()
            typ =' '.join([e.ent_type_ for e in ent])
            nes.append(text +'||' + typ)
        
            
    return nes


def extractNumbers(doc):
    nes=[]
    ents=[]
    for e in doc.ents:
        ents.expand([a for a in e])
        
    for t in doc:
        if t in ents and t.isA:
        if len(ent.text) > 1:
            text= ent.text.lower()
            typ =' '.join([e.ent_type_ for e in ent])
            nes.append(text +'||' + typ)
        
            
    return nes

In [4]:
def preprocess(text):
        #txt=''.join([a+'. ' if not hasEOL(a) else a for a in [ t.strip() for t in text.split('\n')] ])
        txt =re.sub(r'(\?|\!|\.)([^\s])',r'\1 \2',text)
        txt =re.sub(r'(\?|\!|\.)([\.]+)',r'\1 ',text)
        txt = re.sub("amp;nl|amp;",'',txt)
        txt = re.sub("n't",'',txt)
        return txt

def remove_hash(c):
    return re.sub(r'[#|@](\w+)','',c)

def ngrams(d,n, **kwargs):
        text= d.decode('utf-8').strip()
        #text=remove_hash(text)
        text = preprocess(text)
        text=unicode(text)
        doc = nlp(text)
        grams= special_ngrams(doc,n,**kwargs)
        return [ng.text.lower() for ng in grams]

def trigram_pos(d):
    pos=('VERB','NOUN','ADJ','ADP','PROPN','ADV')
    return ngrams(d,3,include_pos_tags=pos)
def trigram(d):
    return ngrams(d,3)
def fourgram(d):
    return ngrams(d,4)
def unigram(d):
    uniqrams=[]
    grams=ngrams(d,1)
    for word in grams:
        if not word in ignore_words and len(word)>1:
            if word=="nt":
                uniqrams.append("n't")
            else:
                uniqrams.append(word)
    return uniqrams
def NER(d):
    doc = getDoc(d)
    return ne(doc)
    
def extract_features(corpus, f):
    j=0
    features=[]
    for d in corpus:
        j =j+1
        result=[i for i in f(d)]
        if result:
            features.append(result)
    return features
def getDoc(txt):
    text=txt.decode('utf-8').strip()
    text = preprocess(text)
    text=unicode(text)
    doc = nlp(text)
    return doc
def docsFromExcel(filename,text_col='customer_verbatim'):
    docFrame = pd.read_excel(filename).head(1000)
    for i, r in docFrame.iterrows():
        utext=r[text_col]
        if not pd.isnull(utext) and isinstance(utext, (str, unicode)) and len(utext)>1:
            yield utext
        
def docsFromText(filename, sep=',',text_col_index=0,skipHeader=False):
    inFile= open(filename)
    if skipHeader:
        inFile.readline()
    for l in inFile:
        fields=l.split(sep)
        text=fields[text_col_index]
        yield text

            
def fromText(filename, delimiter=',',text_col=0, has_header=False,quoting=csv.QUOTE_NONE):
    inFile= open(filename)
    text_col_index=0
    inFile = open(filename, 'rt',encoding='utf8')
    csv_reader = csv.reader(inFile, delimiter=delimiter, quoting=quoting)
    headers=[]
    for r in csv_reader:
        if len(r)>0:
            text=r[text_col_index]
            if isinstance(text, (str, bytes)) and len(text)> 0:
                text=text.strip()
                if text:
                    yield text




def getFiles(path):
    files = [f for f in listdir(path) if isfile(join(path, f))]
    return files



def save_xls(list_dfs, names, xls_path):
    writer = ExcelWriter(xls_path)
    for n, df in enumerate(list_dfs):
        df.to_excel(writer, names[n])
    writer.save()

In [5]:



def genNGram(corpus, min_df=2, max_df=1.0):
    vectorizer = CountVectorizer(analyzer=trigram,  min_df=min_df, max_df=max_df)
    dtm = vectorizer.fit_transform(corpus)
    top_terms = topTerms(dtm,vectorizer.get_feature_names(),cols=['NGRAMS','Frequency'],n=1000)
    return top_terms






                                         


In [6]:
result=[]
path ="C:\\Users\\v617346\\Documents\\Jupyter\\Social\\data\\Weekly_Trigram_Nov6-13\\"
out_path ="C:\\Users\\v617346\\Documents\\Jupyter\\Social\\out\\Weekly Trigrams_20161114.xlsx"
inputFiles =getFiles(path)
for f in inputFiles:
    filename = join(path,f)
    corpus = docsFromText(filename)
    r =genNGram(corpus)
    result.append(r)
sheets = [splitext(s)[0][:30] for i,s in enumerate(inputFiles) ]

save_xls(result,sheets,out_path)

In [ ]:
top_terms.to_excel('C:\\Users\\v617346\\Documents\\Jupyter\\social\\out\\trigram2_20161101.xlsx')

# Linear Discrimant Analysis

In [ ]:
def runDescriminator():
    sdba_corpus =docsFromText(sdba_file)
    sdbd_corpus=docsFromText(sdbd_file)

    sdba_terms =extract_features(sdba_corpus,trigram_pos)
    sdbd_terms =extract_features(sdbd_corpus,trigram_pos)

    return discriminator(sdbd_terms,sdba_terms,['sdbd','sdba'] )

result =runDescriminator()
result
#clear()

# Unsupervised Machine Learning using NMF/LSA /LDA

In [ ]:

from __future__ import absolute_import, division, print_function, unicode_literals

import logging

import numpy as np
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.externals import joblib

    
class TopicModel(object):
    """
    Args:
        model ({'nmf', 'lda', 'lsa'} or ``sklearn.decomposition.<model>``)
        n_topics (int, optional): number of topics in the model to be initialized
        kwargs:
            variety of parameters used to initialize the model; see individual
            sklearn pages for full details

    Raises:
        ValueError: if ``model`` not in ``{'nmf', 'lda', 'lsa'}`` or is not an
            NMF, LatentDirichletAllocation, or TruncatedSVD instance

    Notes:
        - http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html
        - http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html
        - http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html
    """
    def __init__(self, model, n_topics=10, **kwargs):
        if isinstance(model, (NMF, LatentDirichletAllocation, TruncatedSVD)):
            self.model = model
        else:
            self.init_model(model, n_topics=n_topics, **kwargs)

    def init_model(self, model, n_topics=10, **kwargs):
        if model == 'nmf':
            self.model = NMF(
                n_components=n_topics,
                init='nndsvd',
                alpha=kwargs.get('alpha', 0.1),
                l1_ratio=kwargs.get('l1_ratio', 0.5),
                max_iter=kwargs.get('max_iter', 200),
                random_state=kwargs.get('random_state', 1),
                shuffle=kwargs.get('shuffle', False))
        elif model == 'lda':
            self.model = LatentDirichletAllocation(
                n_topics=n_topics,
                max_iter=kwargs.get('max_iter', 10),
                random_state=kwargs.get('random_state', 1),
                learning_method=kwargs.get('learning_method', 'online'),
                learning_offset=kwargs.get('learning_offset', 10.0),
                batch_size=kwargs.get('batch_size', 128),
                n_jobs=kwargs.get('n_jobs', 1))
        elif model == 'lsa':
            self.model = TruncatedSVD(
                n_components=n_topics,
                algorithm=kwargs.get('algorithm', 'randomized'),
                n_iter=kwargs.get('n_iter', 5),
                random_state=kwargs.get('random_state', 1))
        else:
            msg = 'model "{}" invalid; must be {}'.format(      
                model, {'nmf', 'lda', 'lsa'})
            raise ValueError(msg)

    def save(self, filename):
        _ = joblib.dump(self.model, filename, compress=3)
        logger.info('{} model saved to {}'.format(self.model, filename))

    @classmethod
    def load(cls, filename):
        model = joblib.load(filename)
        n_topics = model.n_topics if hasattr(model, 'n_topics') else model.n_components
        return cls(model, n_topics=n_topics)

    def fit(self, doc_term_matrix):
        self.model.fit(doc_term_matrix)

    def partial_fit(self, doc_term_matrix):
        if isinstance(self.model, LatentDirichletAllocation):
            self.model.partial_fit(doc_term_matrix)
        else:
            raise TypeError('only LatentDirichletAllocation models have partial_fit')

    def transform(self, doc_term_matrix):
        return self.model.transform(doc_term_matrix)

    @property
    def n_topics(self):
        try:
            return self.model.n_topics
        except AttributeError:
            return self.model.n_components

    def get_doc_topic_matrix(self, doc_term_matrix, normalize=True):
        """
        Transform a document-term matrix into a document-topic matrix, where rows
        correspond to documents and columns to the topics in the topic model.

        Args:
            doc_term_matrix (array-like or sparse matrix): corpus represented as a
                document-term matrix with shape (n_docs, n_terms); NOTE: LDA expects
                tf-weighting, while NMF and LSA may do better with tfidf-weighting!
            normalize (bool, optional): if True, the values in each row are normalized,
                i.e. topic weights on each document sum to 1

        Returns:
            ``numpy.ndarray``: document-topic matrix with shape (n_docs, n_topics)
        """
        doc_topic_matrix = self.transform(doc_term_matrix)
        if normalize is True:
            return doc_topic_matrix / np.sum(doc_topic_matrix, axis=1, keepdims=True)
        else:
            return doc_topic_matrix

    def top_topic_terms(self, id2term, topics=-1, top_n=10, weights=False):
        """
        Get the top ``top_n`` terms by weight per topic in ``model``.

        Args:
            id2term (list(str) or dict): object that returns the term string corresponding
                to term id ``i`` through ``id2term[i]``; could be a list of strings
                where the index represents the term id, such as that returned by
                ``sklearn.feature_extraction.text.CountVectorizer.get_feature_names()``,
                or a mapping of term id: term string
            topics (int or seq(int), optional): topic(s) for which to return top terms;
                if -1 (default), all topics' terms are returned
            top_n (int, optional): number of top terms to return per topic
            weights (bool, optional): if True, terms are returned with their corresponding
                topic weights; otherwise, terms are returned without weights

        Yields:
            tuple(int, tuple(str)) or tuple(int, tuple((str, float))):
                next tuple corresponding to a topic; the first element is the topic's
                index; if ``weights`` is False, the second element is a tuple of str
                representing the top ``top_n`` related terms; otherwise, the second
                is a tuple of (str, float) pairs representing the top ``top_n``
                related terms and their associated weights wrt the topic; for example::

                    >>> list(TopicModel.top_topic_terms(id2term, topics=(0, 1), top_n=2, weights=False))
                    [(0, ('foo', 'bar')), (1, ('bat', 'baz'))]
                    >>> list(TopicModel.top_topic_terms(id2term, topics=0, top_n=2, weights=True))
                    [(0, (('foo', 0.1415), ('bar', 0.0986)))]
        """
        if topics == -1:
            topics = range(self.n_topics)
        elif isinstance(topics, int):
            topics = (topics,)

        for topic_idx in topics:
            topic = self.model.components_[topic_idx]
            if weights is False:
                yield (topic_idx,
                       tuple(id2term[i] for i in np.argsort(topic)[:-top_n - 1:-1]))
            else:
                yield (topic_idx,
                       tuple((id2term[i], topic[i]) for i in np.argsort(topic)[:-top_n - 1:-1]))

    def top_topic_docs(self, doc_topic_matrix,
                       topics=-1, top_n=10, weights=False):
        """
        Get the top ``top_n`` docs by weight per topic in ``doc_topic_matrix``.

        Args:
            doc_topic_matrix (numpy.ndarray): document-topic matrix with shape
                (n_docs, n_topics), the result of calling
                :func:`get_doc_topic_matrix() <vzta.topic_modeling.get_doc_topic_matrix>`
            topics (seq(int) or int, optional): topic(s) for which to return top docs;
                if -1, all topics' docs are returned
            top_n (int, optional): number of top docs to return per topic
            weights (bool, optional): if True, docs are returned with their corresponding
                (normalized) topic weights; otherwise, docs are returned without weights

        Yields:
            tuple(int, tuple(int)) or tuple(int, tuple(int, float)):
                next tuple corresponding to a topic; the first element is the topic's
                index; if ``weights`` is False, the second element is a tuple of ints
                representing the top ``top_n`` related docs; otherwise, the second
                is a tuple of (int, float) pairs representing the top ``top_n``
                related docs and their associated weights wrt the topic; for example::

                    >>> list(TopicModel.top_doc_terms(dtm, topics=(0, 1), top_n=2, weights=False))
                    [(0, (4, 2)), (1, (1, 3))]
                    >>> list(TopicModel.top_doc_terms(dtm, topics=0, top_n=2, weights=True))
                    [(0, ((4, 0.3217), (2, 0.2154)))]
        """
        if topics == -1:
            topics = range(self.n_topics)
        elif isinstance(topics, int):
            topics = (topics,)

        for topic_idx in topics:
            top_doc_idxs = np.argsort(doc_topic_matrix[:, topic_idx])[:-top_n - 1:-1]
            if weights is False:
                yield (topic_idx,
                       tuple(doc_idx for doc_idx in top_doc_idxs))
            else:
                yield (topic_idx,
                       tuple((doc_idx, doc_topic_matrix[doc_idx, topic_idx]) for doc_idx in top_doc_idxs))

    def top_doc_topics(self, doc_topic_matrix, docs=-1, top_n=3, weights=False):
        """
        Get the top ``top_n`` topics by weight per doc for ``docs`` in ``doc_topic_matrix``.

        Args:
            doc_topic_matrix (numpy.ndarray): document-topic matrix with shape
                (n_docs, n_topics), the result of calling
                :func:`get_doc_topic_matrix() <vzta.topic_modeling.get_doc_topic_matrix>`
            docs (seq(int) or int, optional): docs for which to return top topics;
                if -1, all docs' top topics are returned
            top_n (int, optional): number of top topics to return per doc
            weights (bool, optional): if True, docs are returned with their corresponding
                (normalized) topic weights; otherwise, docs are returned without weights

        Yields:
            tuple(int, tuple(int)) or tuple(int, tuple(int, float)):
                next tuple corresponding to a doc; the first element is the doc's
                index; if ``weights`` is False, the second element is a tuple of ints
                representing the top ``top_n`` related topics; otherwise, the second
                is a tuple of (int, float) pairs representing the top ``top_n``
                related topics and their associated weights wrt the doc; for example::

                    >>> list(TopicModel.top_doc_topics(dtm, docs=(0, 1), top_n=2, weights=False))
                    [(0, (1, 4)), (1, (3, 2))]
                    >>> list(TopicModel.top_doc_topics(dtm, docs=0, top_n=2, weights=True))
                    [(0, ((1, 0.2855), (4, 0.2412)))]
        """
        if docs == -1:
            docs = range(doc_topic_matrix.shape[0])
        elif isinstance(docs, int):
            docs = (docs,)

        for doc_idx in docs:
            top_topic_idxs = np.argsort(doc_topic_matrix[doc_idx, :])[:-top_n - 1:-1]
            if weights is False:
                yield (doc_idx,
                       tuple(topic_idx for topic_idx in top_topic_idxs))
            else:
                yield (doc_idx,
                       tuple((topic_idx, doc_topic_matrix[doc_idx, topic_idx]) for topic_idx in top_topic_idxs))

    def topic_weights(self, doc_topic_matrix):
        """
        Get the overall weight of topics across an entire corpus. Note: Values depend
        on whether topic weights per document in ``doc_topic_matrix`` were normalized,
        or not. I suppose either way makes sense... o_O

        Args:
            doc_topic_matrix (numpy.ndarray): document-topic matrix with shape
                (n_docs, n_topics), the result of calling
                :func:`get_doc_topic_matrix() <vzta.topic_modeling.get_doc_topic_matrix>`

        Returns:
            ``numpy.ndarray``: the ith element is the ith topic's overall weight
        """
        return doc_topic_matrix.sum(axis=0) / doc_topic_matrix.sum(axis=0).sum()




def top_features(matrix,id2term,n=100):
    feature_array = np.array(id2term)
    tfidf_sorting = np.argsort(matrix.toarray()).flatten()[::-1]
    top = feature_array[tfidf_sorting][:n]
    return top


def runModel(model_name, corpus, col_prefix, n_topics=15, n_words=20):
    vectorizer = TfidfVectorizer(analyzer=unigram,  min_df=3, max_df=0.95)
    dtm = vectorizer.fit_transform(corpus)
    
    model = TopicModel(model_name, n_topics=n_topics)
    model.fit(dtm)
    topic_matrix = model.transform(dtm)
    id2terms =vectorizer.get_feature_names()
    
    
    topic_weight=np.sum(topic_matrix, axis=0, keepdims=True)
    allweight = np.sum(topic_weight, axis=1, keepdims=True)[0][0]
    topic_weight_normalized=topic_weight/allweight
    tweights=topic_weight_normalized[0].tolist()
    
    topic_name=[]
    topic_words=[]
    words_col_name=col_prefix+ " Top Words "
    weight_col_name=col_prefix+ " Topic Weight%"
    topic_col_name= col_prefix+" Topics"
    
    for tidx, top_terms in model.top_topic_terms(id2terms, top_n=n_words):
        topic_name.append(top_terms[0]+','+top_terms[1])
        topic_words.append(','.join(top_terms))
    data={words_col_name:topic_words,weight_col_name:tweights}
    dframe =pd.DataFrame(data, columns=data.keys(),index=topic_name)
    dframe.index.name=topic_col_name
    dframe.sort_values(by=weight_col_name,inplace=True, ascending=False)
    dframe[weight_col_name]=dframe[weight_col_name].apply(lambda x: round(x*100,2))
    dframe=dframe[[words_col_name,weight_col_name]]
    return dframe
    

In [ ]:
top_n_words=20
#sdba_corpus =docsFromText(sdba_file)
#sdbd_corpus=docsFromText(sdbd_file)
social_corpus =docsFromText(social_file)

social_topics =runModel('nmf',social_corpus,'social')
#sdbd_df =runModel('lda',sdbd_corpus,'SDBD')



In [ ]:
social_topics.to_excel('C:\\Users\\v617346\\Documents\\Jupyter\\social\\out\\topics_20161101.xlsx')


In [ ]:
social_topics